In [1]:
import pandas as pd
import numpy as np

In [83]:
poetry = pd.read_csv('../data/final_quatrains.csv')

In [72]:
dictionary = pd.read_csv('../data/dictionary_cleaned.csv', index_col=0)

In [84]:
poetry

,text
0,澄潭皎鏡石崔巍萬壑千岩暗綠苔林亭自有幽貞趣況複秋深爽氣來
1,憶昔嬌妃在紫宸鉛華不禦得天真霜綃雖似當時態爭奈嬌波不顧人
2,刻木牽絲作老翁雞皮鶴發與真同須臾弄罷寂無事還似人生一夢中
3,禁苑秋來爽氣多昆明風動起滄波中流簫鼓誠堪賞詎假橫汾發棹歌
4,大殿連雲接爽溪鐘聲還與鼓聲齊長安若問江南事說道風光在水西
...,...
62206,故園東望路漫漫雙袖龍鍾淚不乾馬上相逢無紙筆憑君傳語報平安
62207,黃沙磧裏客行迷四望雲天直下低爲言地盡天還盡行到安西更向西
62208,西向輪臺萬里馀也知鄉信日應疎隴山鸚鵡能言語爲報家人數寄書
62209,朱脣一點桃花殷宿妝嬌羞偏髻鬟細看只似陽臺女醉着莫許歸巫山


In [73]:
dictionary

,char,tone,rime,ipa,tone_class,pinyin,jyutping,hangul,rime_index,meter_class
0,㕙,平,諄,tshwin,L,jùn,㕙,준,6,1
1,㕮,上,麌,pju,X,fǔ,㕮,부,4,0
2,㖃,上,厚,huw,X,hǒu,㖃,후,14,0
3,㘆,上,海,thoj,X,tái,㘆,대,5,0
4,㘱,平,談,tham,L,㘱,㘱,담,16,1
...,...,...,...,...,...,...,...,...,...,...
7975,𩛿,平,清,zjeng,L,𩛿,𩛿,𩛿,12,1
7976,𩜾,平,仙,tsyen,L,𩜾,𩜾,𩜾,7,1
7977,𩥇,去,線,trjen,H,𩥇,𩥇,𩥇,7,0
7978,𩮀,平,東,tsuwng,L,𩮀,𩮀,𩮀,1,1


In [74]:
chars = {dictionary.char[i]: i for i in range(0,len(dictionary))}
char_dict = {i: {'char': dictionary.char[i],
                 'tone': dictionary.tone_class[i],
                 'rime': dictionary.rime_index[i],
                 'meter': dictionary.meter_class[i]
                 } for i in range(0,len(dictionary))}

In [97]:
meters = [
    [1,1,0,0,1,1,0,
     0,0,1,1,0,0,1,
     0,0,1,1,0,0,1,
     1,1,0,0,0,1,1],
    [0,0,1,1,1,0,0,
     1,1,0,0,0,1,1,
     1,1,0,0,1,1,0,
     0,0,1,1,0,0,1],
    [1,1,0,0,0,1,1,
     0,0,1,1,0,0,1,
     0,0,1,1,1,0,0,
     1,1,0,0,0,1,1],
    [0,0,1,1,0,0,1,
     1,1,0,0,0,1,1,
     1,1,0,0,1,1,0,
     0,0,1,1,0,0,1],
    [0,0,1,1,0,0,1,
     1,1,0,0,0,1,1,
     1,1,0,0,1,1,0,
     0,0,1,1,0,0,1],
    [1,1,0,0,0,1,1,
     0,0,1,1,0,0,1,
     0,0,1,1,1,0,0,
     1,1,0,0,0,1,1],
    [0,0,1,1,1,0,0,
     1,1,0,0,0,1,1,
     1,1,0,0,1,1,0,
     0,0,1,1,0,0,1],
    [1,1,0,0,1,1,0,
     0,0,1,1,0,0,1,
     0,0,1,1,1,0,0,
     1,1,0,0,0,1,1]
]

In [19]:
def text_tokenizer(text):
    return [chars[i] for i in text]

In [38]:
def token_to_text(tokens):
    return ''.join([char_dict[i]['char'] for i in tokens])

In [115]:
def rhyme_checker(tokens):
    if char_dict[tokens[13]]['rime'] == char_dict[tokens[27]]['rime']:
        return 1
    else:
        return 0

In [99]:
def show_meter(tokens):
    return [char_dict[i]['meter'] for i in tokens]

In [101]:
def meter_score(tokens):
    meter = [char_dict[i]['meter'] for i in tokens]
    scores = []
    for n in meters:
        scores.append(sum([int(meter[num]==n[num]) for num in n]))
    return max(scores)/28

In [113]:
poetry['tokens'] = poetry['text'].apply(text_tokenizer)

In [114]:
poetry['meter_score'] = poetry['tokens'].apply(meter_score)

In [116]:
poetry['rhyme'] = poetry['tokens'].apply(rhyme_checker)

In [120]:
poetry['target'] = 1

In [119]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, LSTM, GRU
from sklearn.model_selection import train_test_split

In [145]:
X = poetry.drop(columns='text')
X.tokens = X.tokens.apply(np.array)
X = np.array(X)
y = np.array(poetry.target)

In [148]:
X

array([[array([3497, 3485, 4137, 7002, 4278, 1538, 1605, 5528, 1129,  627, 1504,
               2590, 4856, 5394, 2710,  140, 5288, 2638, 1693, 6303, 6395, 3204,
               5994, 4433, 3336, 3722, 3133,  231])                             ,
        1.0, 1, 1],
       [array([2064, 2534, 1323, 1208,  999, 4802, 1392, 6911, 5504,   74, 4412,
               1830, 1165, 4197, 7232, 4848, 7198,  191, 4042, 2555, 2015, 3713,
               1180, 1323, 3222,   74, 7383,  147])                             ,
        1.0, 1, 1],
       [array([ 514, 2654, 3752, 4841,  211, 5096, 5066, 7204, 4149, 7779, 4126,
               5304, 4197,  714, 7335, 5297, 1756, 5024, 1396, 3629,  120, 6711,
                191,  147, 3995,   67, 1161,   89])                             ,
        1.0, 1, 1],
       ...,
       [array([6037,  719, 6556, 5294, 5528, 6875, 7478,  111, 4269, 6774,  266,
               2512, 2070, 4053, 7178, 1487, 7816, 7751, 5195, 6080, 6150, 3716,
               1076, 1391,  147, 2

In [118]:
discriminator = Sequential()
discriminator. add

In [112]:
model.add(meter_score)

TypeError: The added layer must be an instance of class Layer. Received: layer=<function meter_score at 0x0000021D6DB5DB80> of type <class 'function'>.